## Error Correction

In [1]:
import plotnine as p9
import pandas as pd
import scipy.stats as stats
import numpy as np
import statsmodels as sm

In [2]:
data = pd.read_csv('/Users/xingkaiwu/OneDrive - York University/MBAN Winter/MBAN 6400/Datacamp Experiment Design/olyathswim.csv')
athletes = data[(data['Height'].isna()== False) & (data['Team']=='United States') & (data['Year'].isin([1924,1952,2016])) & (data['Sex']=='M')]

### Bonferroni correction

In [5]:
# Perform three two-sample t-tests 
t_result_1924v2016= stats.ttest_ind(athletes[athletes['Year']==1924].Height, athletes[athletes['Year']==2016].Height)
t_result_1952v2016= stats.ttest_ind(athletes[athletes['Year']==1952].Height, athletes[athletes['Year']==2016].Height)
t_result_1924v1952= stats.ttest_ind(athletes[athletes['Year']==1924].Height, athletes[athletes['Year']==1952].Height)

# Create an array of p-value results
pvals_array = [t_result_1924v2016[1], t_result_1952v2016[1], t_result_1924v1952[1]]
print(pvals_array)

# Perform Bonferroni correction
adjustedvalues = sm.stats.multitest.multipletests(pvals_array, alpha=0.05, method='b')
print(adjustedvalues)

[2.0995273280295586e-16, 4.484419167217926e-05, 0.00013562429334889781]
(array([ True,  True,  True]), array([6.29858198e-16, 1.34532575e-04, 4.06872880e-04]), 0.016952427508441503, 0.016666666666666666)


### Šídák correction

In [21]:
athletes = data[(data['Height'].isna()== False) & (data['Team']=='United States') & ((data['Event'].str.endswith('100 metres')) | (data['Event'].str.endswith("Men's High Jump")) | (data['Event'].str.endswith('Marathon'))) & (data['Sex']=='M')]
athletes['Event_1'] = athletes['Event'].map(lambda x: '100 meters' if (x.find('100 metres')!= -1) else ('High Jump' if (x.find('High Jump')!= -1) else 'Marathon'))

/var/folders/9s/zz7k8h2949d__gv0rmbn3yj80000gn/T/ipykernel_91968/2286331211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
athletes.Event_1.value_counts()

100 meters    103
Marathon       99
High Jump      88
Name: Event_1, dtype: int64

In [26]:
# Perform Pearson correlations
pearson100 = stats.pearsonr(athletes[athletes.Event_1 == '100 meters'].Height, athletes[athletes.Event_1 == '100 meters'].Year)
pearsonHigh = stats.pearsonr(athletes[athletes.Event_1 == 'High Jump'].Height, athletes[athletes.Event_1 == 'High Jump'].Year)
pearsonMara = stats.pearsonr(athletes[athletes.Event_1 == 'Marathon'].Height, athletes[athletes.Event_1 == 'Marathon'].Year)

# Create array of p-values
pvals_array = [pearson100[1],pearsonHigh[1],pearsonMara[1]]
print(pvals_array)

# Perform Šídák correction
adjustedvalues=  sm.stats.multitest.multipletests(pvals_array, method= 's')
print(adjustedvalues)

[0.00991263413561775, 9.848384269358676e-07, 2.03170170479967e-05]
(array([ True,  True,  True]), array([2.94440955e-02, 2.95451237e-06, 6.09498128e-05]), 0.016952427508441503, 0.016666666666666666)
